**Engy-4390** Nuclear Systems Design and Analysis, UMass Lowell, Chem. Eng. Nuclear Program Fall 2024; Prof. V. F. de Almeida

# Discussion On Test Four

**Student(s):**  Kyle Mills and Billy Manco and Donaita Anglade <br>
**Course:** Engy-4390: Nuclear Systems Design and Analysis  <br>
**Mentor and Collaborator:** Subash Sharma

UMass Lowell, Dept. of Chemical Engineering, Nuclear Energy Program 
<br>
**21 Oct 2024**

---
## Table of Contents<a id="toc"></a>
* [Definitions](#defs)
* [Setup](#setup)
* [Test](#test)
---

## [Definition of the necessary functions](#toc)<a id="defs"></a>

## [Setup](#toc)<a id="setup"></a>

## [Test](#toc)<a id="test"></a>